데이터 마운트, 모듈

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
# 확인 필수!!!!
#  드라이브 마운트 후 진행해야 합니다.
DATA_PATH=""

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
#기계학습 모델 생성, 학습, 평가
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score,precision_score,mean_squared_error,mean_absolute_error,r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree # 의사결정나무 규칙 시각화용
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
import sklearn.datasets as d
from tqdm import tqdm # 오 이거 신기하다.
# 시각화
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
# 한글폰트 지원

In [ ]:
obe = pd.read_csv("/content/drive/MyDrive/멀티캠퍼스_머신러닝/세미 프로젝트 비만/Project/Data/ObesityDataSet_OneHot.csv")

#############랜덤 포레스트

In [ ]:
# 스케일링
std_scaler = StandardScaler()
mm_scaler = MinMaxScaler()
original_labeled_Scaled = obe
cont_vars = ['Age', 'Height', 'Weight']
ordinal_vars = ['CAEC_no','CAEC_Sometimes',	'CAEC_Frequently','CAEC_Always', 'CALC_no','CALC_Sometimes','CALC_Frequently','CALC_Always', 'MTRANS_CarBike','MTRANS_Public','MTRANS_BikeWalking','FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
original_labeled_Scaled[cont_vars] = std_scaler.fit_transform(original_labeled_Scaled[cont_vars])
original_labeled_Scaled[ordinal_vars] = mm_scaler.fit_transform(original_labeled_Scaled[ordinal_vars])
# 트레인/ 테스트를 분리하자 아 validation은 랜덤 포레스트에서 권장하지 않는다고 함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=obe['NObeyesdad']) #,stratify=obe['NObeyesdad']
# 데이터 컬럼 단위 정규화하기
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train.values)
# X_test = scaler.transform(X_test.values)

# print(f'{X_train.shape}')
# print(f'{X_train.shape}')


In [ ]:
# # 단순하게 모델링하면..
# obe_r = RandomForestClassifier(n_estimators=5, random_state=42)
# obe_r.fit(X_train,y_train)
# # train 데이터에 대한 성능
# y_pred = obe_r.predict(X_train)
# acc = accuracy_score(y_train, y_pred = y_pred)
# print("train set에 대한 성능")
# print(f"정확도:{acc:0.4f}")

# # Test set에 대한 성능
# y_pred = obe_r.predict(X_test)
# acc = accuracy_score(y_true = y_test, y_pred=y_pred)
# print("test set에 대한 성능")
# print(f"정확도:{acc:0.4f}")
# #

train set에 대한 성능
정확도:0.9882
test set에 대한 성능
정확도:0.8983


In [ ]:
#
# 익태님: 스케일링하고 데이터 쪼개는 게 오류가 안 난다.
# 은아님: 그리드서치 범위 잡아놓고 어떤 범위에서 예측 성능이 좋을 지 생각해보기, 범위 좁혀보기, 특성 중요도, 트리 시각화.


In [ ]:
# 전체 하이퍼파라미터
# 1. n_estimators - 결정트리의 갯수를 지정, 디폴트값은 보통 10-> 무작정 트리 갯수를 늘리면 성능 좋아지는 것 대비 시간이 걸림
# 2. min_samples_split - 노드를 분할하기 위한 최소한의 샘플 데이터수, 디폴트는 2 -> 작게 설정할수록 분할 노드가 많아져 과적합 가능성 증가
# 3. min_samples_leaf- 리프노드가 되기 위해 최소한의 샘플 데이터수,min_samples_split과 함께 과적합 제어, 불균형 데이터의 경우 특정 클래스의 데이터가 극도로 작을 수 있으므로 작게 설정하는 것이 좋음.
# 4. max_features- 최적의 분할을 위해 고려한 최대 피처 갯수, 디폴트는 auto, int로 지정->피처개수 float로 지정-> 비중,
# sqrt 또는 auto: 전체 피처 중 루트(피처개수)만큼 선정 , log: 전체 피처 중 log2만큼 선정
# 5. max_depth - 트리의 최대 깊이, default = None -> 완벽하게 클래스 값이 결정될 때까지 분할, 깊이가 깊어지면 과적합 될 수 있으므로 적절히 제어해야함.
# 6. max_leaf_nodes- 리프노드의 최대개수
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# 모델 정의
rf_classifier = RandomForestClassifier()

# 그리드 서치를 위한 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [10, 50, 100],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_depth': [None, 10, 20, 30],  # 트리의 최대 깊이

}

# 그리드 서치 객체 생성
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5)

# 데이터에 대해 그리드 서치 수행
grid_search.fit(X_train, y_train)

# 최적의 모델 및 하이퍼파라미터 출력
print("Best parameters found: ", grid_search.best_params_)
print("Best estimator found: ", grid_search.best_estimator_)


Best parameters found:  {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best estimator found:  RandomForestClassifier(max_depth=20)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 최적의 하이퍼파라미터 값을 적용하여 모델 정의
obe_r_optimized = RandomForestClassifier(n_estimators=grid_search.best_params_['n_estimators'],
                                         min_samples_split=grid_search.best_params_['min_samples_split'],
                                         min_samples_leaf=grid_search.best_params_['min_samples_leaf'],

                                         max_depth=grid_search.best_params_['max_depth'],

                                         n_jobs=-1, random_state=42)

# 모델 학습
obe_r_optimized.fit(X_train, y_train)

# Train 데이터에 대한 성능 측정
y_pred_train = obe_r_optimized.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
print("Train set에 대한 성능:")
print(f"정확도: {train_accuracy:.4f}")

# Test 데이터에 대한 성능 측정
y_pred_test = obe_r_optimized.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test set에 대한 성능:")
print(f"정확도: {test_accuracy:.4f}")

# 추가적인 성능 측정
my_accuracy = accuracy_score(y_true=y_test, y_pred=y_pred_test)
my_precision = precision_score(y_true=y_test, y_pred=y_pred_test, average='macro')
my_recall = recall_score(y_true=y_test, y_pred=y_pred_test, average='macro')
f1 = f1_score(y_true=y_test, y_pred=y_pred_test, average='macro')

# 결과 출력
print(f"정확도: {my_accuracy:.4f}")
print(f"정밀도: {my_precision:.4f}")
print(f"재현율: {my_recall:.4f}")
print(f"F1 점수: {f1:.4f}")


Train set에 대한 성능:
정확도: 1.0000
Test set에 대한 성능:
정확도: 0.9409
정확도: 0.9409
정밀도: 0.9449
재현율: 0.9383
F1 점수: 0.9398


In [ ]:
# # 측정기
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 테스트 데이터에 대한 예측
my_y_pred = obe_r_optimized.predict(X_test)

# 테스트 데이터에 대한 평가 지표 계산
my_accuracy = accuracy_score(y_true=y_test, y_pred=my_y_pred)
my_precision = precision_score(y_true=y_test, y_pred=my_y_pred, average='macro')
my_recall = recall_score(y_true=y_test, y_pred=my_y_pred, average='macro')
my_f1 = f1_score(y_true=y_test, y_pred=my_y_pred, average='macro')

# 훈련 데이터에 대한 예측
train_pred = obe_r_optimized.predict(X_train)

# 훈련 데이터에 대한 F1 점수 계산
train_f1 = f1_score(y_true=y_train, y_pred=train_pred, average='macro')

# 출력
print(f"정확도: {my_accuracy:.4f}")
print(f"정밀도: {my_precision:.4f}")
print(f"재현율: {my_recall:.4f}")
print(f"F1 점수: {my_f1:.4f}")

# 훈련 데이터에 대한 F1 점수 출력
print(f"훈련 데이터에 대한 F1 점수: {train_f1:.4f}")



정확도: 0.9409
정밀도: 0.9449
재현율: 0.9383
F1 점수: 0.9398
훈련 데이터에 대한 F1 점수: 1.0000


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 특성 중요도 가져오기
importances = obe_r.feature_importances_

# 특성 이름 가져오기
feature_names = X.columns

# 특성 중요도를 데이터프레임으로 만들기
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# 특성 중요도 순으로 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# 특성 중요도 시각화
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()
